In [ ]:
USE ROLE ROLE_TEAM_226;
USE WAREHOUSE Animal_Task_WH;
USE DATABASE DB_TEAM_226;





In [ ]:
CREATE SCHEMA IF NOT EXISTS BRONZE;

In [ ]:
CREATE OR REPLACE TABLE BRONZE.FASHION_BRONZE_RAW (
  ROW_IDX STRING,
  BRANDNAME STRING,
  DEATILS STRING,
  SIZES STRING,
  MRP STRING,
  SELLPRICE STRING,
  DISCOUNT STRING,
  CATEGORY STRING,
  _INGESTION_TIMESTAMP TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP()
);





In [ ]:
CREATE OR REPLACE STAGE BRONZE.FASHION_STAGE;


In [ ]:
SELECT $1, $2, $3, $4, $5, $6, $7, $8
FROM @BRONZE.FASHION_STAGE (FILE_FORMAT => BRONZE.FASHION_CSV_FORMAT)
LIMIT 5;



In [ ]:
SELECT $1,$2,$3,$4,$5,$6,$7,$8
FROM @BRONZE.FASHION_STAGE (FILE_FORMAT => BRONZE.FASHION_CSV_FORMAT)
LIMIT 5;


In [ ]:
COPY INTO BRONZE.FASHION_BRONZE_RAW
  (ROW_IDX, BRANDNAME, DEATILS, SIZES, MRP, SELLPRICE, DISCOUNT, CATEGORY)   -- <--- map only these 8
FROM @BRONZE.FASHION_STAGE
FILE_FORMAT = (FORMAT_NAME = 'BRONZE.FASHION_CSV_FORMAT')
ON_ERROR = 'ABORT_STATEMENT'
FORCE = TRUE;


In [ ]:
SELECT * FROM BRONZE.FASHION_BRONZE_RAW LIMIT 5;


In [ ]:
SELECT COUNT(*) AS rows_loaded FROM BRONZE.FASHION_BRONZE_RAW;
SELECT ROW_IDX, BRANDNAME, MRP, SELLPRICE, CATEGORY, _INGESTION_TIMESTAMP
FROM BRONZE.FASHION_BRONZE_RAW
LIMIT 10;


In [ ]:
SELECT COUNT(*) FROM BRONZE.FASHION_BRONZE_RAW;


Silver Layer:

Task 3-6:


In [ ]:
USE ROLE ROLE_TEAM_226;
USE WAREHOUSE Animal_Task_WH;
USE DATABASE DB_TEAM_226;

-- Create Silver Schema
CREATE SCHEMA IF NOT EXISTS SILVER;

In [ ]:
-- 1. Create DIM_BRAND
CREATE OR REPLACE TABLE SILVER.DIM_BRAND (
    BRAND_ID INT IDENTITY(1,1) PRIMARY KEY,
    BRAND_NAME STRING UNIQUE
);

-- Populate DIM_BRAND with distinct values from Bronze
INSERT INTO SILVER.DIM_BRAND (BRAND_NAME)
SELECT DISTINCT BRANDNAME 
FROM BRONZE.FASHION_BRONZE_RAW 
WHERE BRANDNAME IS NOT NULL;


-- 2. Create DIM_CATEGORY
CREATE OR REPLACE TABLE SILVER.DIM_CATEGORY (
    CATEGORY_ID INT IDENTITY(1,1) PRIMARY KEY,
    CATEGORY_NAME STRING UNIQUE
);

-- Populate DIM_CATEGORY with distinct values from Bronze
INSERT INTO SILVER.DIM_CATEGORY (CATEGORY_NAME)
SELECT DISTINCT CATEGORY 
FROM BRONZE.FASHION_BRONZE_RAW 
WHERE CATEGORY IS NOT NULL;

In [ ]:
CREATE OR REPLACE TABLE SILVER.FACT_FASHION (
    PRODUCT_KEY STRING, -- Original Row Index
    BRAND_ID INT,
    CATEGORY_ID INT,
    DETAILS STRING,
    SIZES STRING,
    MRP FLOAT,
    SELL_PRICE FLOAT,
    DISCOUNT_PERCENT STRING, -- Keeping as string or float depending on raw format
    _INGESTION_TIMESTAMP TIMESTAMP_LTZ DEFAULT CURRENT_TIMESTAMP(),
    CONSTRAINT FK_BRAND FOREIGN KEY (BRAND_ID) REFERENCES SILVER.DIM_BRAND(BRAND_ID),
    CONSTRAINT FK_CAT FOREIGN KEY (CATEGORY_ID) REFERENCES SILVER.DIM_CATEGORY(CATEGORY_ID)
);

In [ ]:
-- INSERT INTO SILVER.FACT_FASHION (
--     PRODUCT_KEY, 
--     BRAND_ID, 
--     CATEGORY_ID, 
--     DETAILS, 
--     SIZES, 
--     MRP, 
--     SELL_PRICE, 
--     DISCOUNT_PERCENT
-- )
-- SELECT 
--     b.ROW_IDX,
--     db.BRAND_ID,
--     dc.CATEGORY_ID,
--     b.DEATILS, -- Mapping original column with typo
--     b.SIZES,
--     -- Cleaning Price Columns: Remove non-numeric chars (except decimal) and cast to Float
--     TRY_CAST(REGEXP_REPLACE(b.MRP, '[^0-9.]', '') AS FLOAT),
--     TRY_CAST(REGEXP_REPLACE(b.SELLPRICE, '[^0-9.]', '') AS FLOAT),
--     b.DISCOUNT
-- FROM BRONZE.FASHION_BRONZE_RAW b
-- LEFT JOIN SILVER.DIM_BRAND db ON b.BRANDNAME = db.BRAND_NAME
-- LEFT JOIN SILVER.DIM_CATEGORY dc ON b.CATEGORY = dc.CATEGORY_NAME;

INSERT INTO SILVER.FACT_FASHION (
    PRODUCT_KEY, 
    BRAND_ID, 
    CATEGORY_ID, 
    DETAILS, 
    SIZES, 
    MRP, 
    SELL_PRICE,      -- <--- Data lands in this column name
    DISCOUNT_PERCENT
)
SELECT 
    b.ROW_IDX,
    db.BRAND_ID,
    dc.CATEGORY_ID,
    b.DEATILS,
    b.SIZES,
    TRY_CAST(REGEXP_REPLACE(b.MRP, '[^0-9.]', '') AS FLOAT),
    -- We calculate the value here. The name 'CLEAN_SELL_PRICE' is temporary.
    TRY_CAST(REGEXP_REPLACE(b.SELLPRICE, '[^0-9.]', '') AS FLOAT) AS CLEAN_SELL_PRICE, 
    b.DISCOUNT
FROM BRONZE.FASHION_BRONZE_RAW b
LEFT JOIN SILVER.DIM_BRAND db ON b.BRANDNAME = db.BRAND_NAME
LEFT JOIN SILVER.DIM_CATEGORY dc ON b.CATEGORY = dc.CATEGORY_NAME
-- FILTER: We repeat the calculation here to filter out the bad rows
WHERE TRY_CAST(REGEXP_REPLACE(b.SELLPRICE, '[^0-9.]', '') AS FLOAT) IS NOT NULL;

In [ ]:
-- Check row counts
SELECT COUNT(*) as SILVER_ROW_COUNT FROM SILVER.FACT_FASHION;

-- Preview Cleaned Data
-- SELECT * FROM SILVER.FACT_FASHION LIMIT 10;

In [ ]:
SELECT COUNT(*) FROM BRONZE.FASHION_BRONZE_RAW;


In [ ]:
select * from BRONZE.FASHION_BRONZE_RAW
limit 10;

In [ ]:
select * from SILVER.FACT_FASHION
limit 10;

The 3 Use Cases:

1. Brand Performance Report: Which brands offer the highest discounts? What is the average price point per brand? This helps the business decide which brands are "Premium" vs. "Budget."


2. Category Inventory Summary: Which categories (e.g., "Western Wear," "Lingerie") have the most stock and the highest total value? This helps with inventory planning.


3. Price Tier Analysis: How many products fall into "Budget," "Mid-Range," or "Premium" price buckets? This helps understand the target customer demographic.

Gold Layer:
Task 7 to 12

In [ ]:
-- Create Gold Schema
CREATE SCHEMA IF NOT EXISTS GOLD;

-- Use Case 1: Brand Performance Report
-- Aggregating average price and discount by Brand
CREATE OR REPLACE TABLE GOLD.BRAND_PERFORMANCE AS
SELECT 
    b.BRAND_NAME,
    COUNT(f.PRODUCT_KEY) AS TOTAL_PRODUCTS,
    ROUND(AVG(f.MRP), 2) AS AVG_MRP,
    ROUND(AVG(f.SELL_PRICE), 2) AS AVG_SELL_PRICE,
    -- Extracting numeric value from Discount string if needed, or just counting
    COUNT(CASE WHEN f.DISCOUNT_PERCENT IS NOT NULL THEN 1 END) AS DISCOUNTED_ITEMS_COUNT
FROM SILVER.FACT_FASHION f
JOIN SILVER.DIM_BRAND b ON f.BRAND_ID = b.BRAND_ID
GROUP BY b.BRAND_NAME
ORDER BY TOTAL_PRODUCTS DESC;

-- Use Case 2: Category Inventory Summary
-- Calculating total inventory value per Category
CREATE OR REPLACE TABLE GOLD.CATEGORY_INVENTORY AS
SELECT 
    c.CATEGORY_NAME,
    COUNT(f.PRODUCT_KEY) AS STOCK_COUNT,
    ROUND(SUM(f.SELL_PRICE), 2) AS TOTAL_INVENTORY_VALUE,
    ROUND(AVG(f.SELL_PRICE), 2) AS AVG_ITEM_VALUE
FROM SILVER.FACT_FASHION f
JOIN SILVER.DIM_CATEGORY c ON f.CATEGORY_ID = c.CATEGORY_ID
GROUP BY c.CATEGORY_NAME
ORDER BY TOTAL_INVENTORY_VALUE DESC;

-- Use Case 3: Price Tier Analysis
-- Bucketing products into Budget, Mid-Range, and Premium
CREATE OR REPLACE TABLE GOLD.PRICE_TIER_ANALYSIS AS
SELECT 
    CASE 
        WHEN SELL_PRICE < 1000 THEN 'Budget'
        WHEN SELL_PRICE BETWEEN 1000 AND 3000 THEN 'Mid-Range'
        WHEN SELL_PRICE > 3000 THEN 'Premium'
        ELSE 'Unknown'
    END AS PRICE_TIER,
    COUNT(*) AS PRODUCT_COUNT,
    ROUND(AVG(SELL_PRICE), 2) AS AVG_PRICE_IN_TIER
FROM SILVER.FACT_FASHION
GROUP BY 1
ORDER BY AVG_PRICE_IN_TIER;

-- View Gold Tables to verify
SELECT * FROM GOLD.BRAND_PERFORMANCE LIMIT 5;
SELECT * FROM GOLD.CATEGORY_INVENTORY LIMIT 5;
SELECT * FROM GOLD.PRICE_TIER_ANALYSIS;

In [ ]:
-- TASK 8 & 9: Load the incremental file (FIXED)
COPY INTO BRONZE.FASHION_BRONZE_RAW
  (ROW_IDX, BRANDNAME, DEATILS, SIZES, MRP, SELLPRICE, DISCOUNT, CATEGORY) -- <--- THIS LIST IS REQUIRED
FROM @BRONZE.FASHION_STAGE
FILES = ('fashion_new.csv')
FILE_FORMAT = (FORMAT_NAME = 'BRONZE.FASHION_CSV_FORMAT')
ON_ERROR = 'ABORT_STATEMENT'
FORCE = TRUE;

In [ ]:
-- TASK 9: Implement mechanism to load incremental data into Silver and Gold

-- 9a. Update Dimensions (Load new Brands and Categories if they don't exist)
INSERT INTO SILVER.DIM_BRAND (BRAND_NAME)
SELECT DISTINCT BRANDNAME
FROM BRONZE.FASHION_BRONZE_RAW
WHERE BRANDNAME IS NOT NULL
  AND BRANDNAME NOT IN (SELECT BRAND_NAME FROM SILVER.DIM_BRAND);

INSERT INTO SILVER.DIM_CATEGORY (CATEGORY_NAME)
SELECT DISTINCT CATEGORY
FROM BRONZE.FASHION_BRONZE_RAW
WHERE CATEGORY IS NOT NULL
  AND CATEGORY NOT IN (SELECT CATEGORY_NAME FROM SILVER.DIM_CATEGORY);

-- 9b. Update Fact Table (Insert only NEW records based on ROW_IDX/PRODUCT_KEY)
INSERT INTO SILVER.FACT_FASHION (
    PRODUCT_KEY, BRAND_ID, CATEGORY_ID, DETAILS, SIZES, MRP, SELL_PRICE, DISCOUNT_PERCENT
)
SELECT
    b.ROW_IDX,
    db.BRAND_ID,
    dc.CATEGORY_ID,
    b.DEATILS,
    b.SIZES,
    TRY_CAST(REGEXP_REPLACE(b.MRP, '[^0-9.]', '') AS FLOAT),
    TRY_CAST(REGEXP_REPLACE(b.SELLPRICE, '[^0-9.]', '') AS FLOAT),
    b.DISCOUNT
FROM BRONZE.FASHION_BRONZE_RAW b
JOIN SILVER.DIM_BRAND db ON b.BRANDNAME = db.BRAND_NAME
JOIN SILVER.DIM_CATEGORY dc ON b.CATEGORY = dc.CATEGORY_NAME
WHERE b.ROW_IDX NOT IN (SELECT PRODUCT_KEY FROM SILVER.FACT_FASHION);

-- 9c. Refresh Gold Layer (Re-calculate aggregates to include new data)
-- We use CREATE OR REPLACE to fully refresh the reports
CREATE OR REPLACE TABLE GOLD.BRAND_PERFORMANCE AS
SELECT
    b.BRAND_NAME,
    COUNT(f.PRODUCT_KEY) AS TOTAL_PRODUCTS,
    ROUND(AVG(f.MRP), 2) AS AVG_MRP,
    ROUND(AVG(f.SELL_PRICE), 2) AS AVG_SELL_PRICE,
    COUNT(CASE WHEN f.DISCOUNT_PERCENT IS NOT NULL THEN 1 END) AS DISCOUNTED_ITEMS_COUNT
FROM SILVER.FACT_FASHION f
JOIN SILVER.DIM_BRAND b ON f.BRAND_ID = b.BRAND_ID
GROUP BY b.BRAND_NAME;

CREATE OR REPLACE TABLE GOLD.CATEGORY_INVENTORY AS
SELECT
    c.CATEGORY_NAME,
    COUNT(f.PRODUCT_KEY) AS TOTAL_STOCK,
    SUM(f.SELL_PRICE) AS TOTAL_INVENTORY_VALUE
FROM SILVER.FACT_FASHION f
JOIN SILVER.DIM_CATEGORY c ON f.CATEGORY_ID = c.CATEGORY_ID
GROUP BY c.CATEGORY_NAME;

In [ ]:
-- TASK 10: Record comparison/verification
-- 1. Check Row Counts (Should be higher than your initial load)
SELECT 'BRONZE' AS LAYER, COUNT(*) AS COUNT FROM BRONZE.FASHION_BRONZE_RAW
UNION ALL
SELECT 'SILVER', COUNT(*) FROM SILVER.FACT_FASHION
UNION ALL
SELECT 'GOLD_BRAND_RPT', COUNT(*) FROM GOLD.BRAND_PERFORMANCE;

-- 2. Specific Data Check
-- Verify that your new brand (e.g., 'NEW_BRAND_XYZ') appears in the Gold report
SELECT *
FROM GOLD.BRAND_PERFORMANCE
WHERE BRAND_NAME IN ('NEW_BRAND_XYZ', 'Zara'); -- Adjust names based on your csv file

In [ ]:
USE ROLE ROLE_TEAM_226;
USE WAREHOUSE Animal_Task_WH;
USE DATABASE DB_TEAM_226;
USE SCHEMA BRONZE;


In [ ]:
-- TASK 11: Run 2 AI/ML functions and store output as additional columns
-- We use 'UPDATE' to calculate the values for the new rows you just loaded

-- 1. Ensure the columns exist (if you haven't created them yet)
ALTER TABLE BRONZE.FASHION_BRONZE_RAW ADD COLUMN IF NOT EXISTS DETAIL_SENTIMENT FLOAT;
ALTER TABLE BRONZE.FASHION_BRONZE_RAW ADD COLUMN IF NOT EXISTS DETAIL_SUMMARY STRING;

-- 2. Run the AI functions to populate the columns
UPDATE BRONZE.FASHION_BRONZE_RAW
SET
    DETAIL_SENTIMENT = SNOWFLAKE.CORTEX.SENTIMENT(DEATILS),
    DETAIL_SUMMARY = SNOWFLAKE.CORTEX.SUMMARIZE(DEATILS)
WHERE DETAIL_SENTIMENT IS NULL; -- Only process rows that need it

In [ ]:
-- CHECK 1: Did the new rows make it to Bronze?
-- Look for the row with ID '99991' (or whatever ID was in your CSV)
SELECT * FROM BRONZE.FASHION_BRONZE_RAW WHERE ROW_IDX = '99991';

-- CHECK 2: Do the Dimensions match?
-- Run this to see if the Join works manually
SELECT
    b.BRANDNAME as BRONZE_BRAND,
    db.BRAND_NAME as SILVER_BRAND
FROM BRONZE.FASHION_BRONZE_RAW b
LEFT JOIN SILVER.DIM_BRAND db ON b.BRANDNAME = db.BRAND_NAME
WHERE b.ROW_IDX = '99991';

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE BRONZE_SEARCH_SVC
  ON DEATILS                         -- this is your descriptive text column
  ATTRIBUTES BRANDNAME, CATEGORY     -- columns you may want to filter/return
  WAREHOUSE = Animal_Task_WH
  TARGET_LAG = '1 minute'
AS (
  SELECT
      ROW_IDX,
      BRANDNAME,
      CATEGORY,
      DEATILS
  FROM BRONZE.FASHION_BRONZE_RAW
);


In [ ]:
SELECT
  PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'DB_TEAM_226.BRONZE.BRONZE_SEARCH_SVC',
      '{
         "query": "summer cotton clothes",
         "columns": [
           "ROW_IDX",
           "BRANDNAME",
           "DEATILS",
           "CATEGORY"
         ],
         "limit": 5
       }'
    )
  )['results'] AS results;


In [ ]:
SELECT
  PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'DB_TEAM_226.BRONZE.BRONZE_SEARCH_SVC',
      '{
         "query": "jacket",
         "columns": [
           "ROW_IDX",
           "BRANDNAME",
           "DEATILS",
           "CATEGORY"
         ],
         "limit": 5
       }'
    )
  )['results'] AS results;


In [ ]:
SELECT
  PARSE_JSON(
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'DB_TEAM_226.BRONZE.BRONZE_SEARCH_SVC',
      '{
         "query": "shirt",
         "columns": [
           "ROW_IDX",
           "BRANDNAME",
           "DEATILS",
           "CATEGORY"
         ],
         "limit": 5
       }'
    )
  )['results'] AS results;


In [ ]:
SHOW TABLES IN SCHEMA SILVER;


In [ ]:
DESC TABLE DB_TEAM_226.SILVER.FACT_FASHION;


In [ ]:
DESC TABLE DB_TEAM_226.SILVER.DIM_BRAND;


In [ ]:
DESC TABLE DB_TEAM_226.SILVER.DIM_CATEGORY;


In [ ]:
CREATE OR REPLACE STAGE semantic_stage;


In [ ]:
# Import necessary libraries
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session

# 1. Get the current active Snowflake session
session = get_active_session()

# Set the title of the Dashboard
st.title("Team 226: Fashion Analytics Dashboard")
st.write("Overview of Brand Performance, Inventory, and Price Tiers based on Gold Layer data.")

# ---------------------------------------------------------
# VISUALIZATION 1: Brand Performance (Top 10 Brands by Volume)
# ---------------------------------------------------------
st.header("1. Brand Performance")
st.write("Top 10 Brands by Average selling price")

# Query the Gold Table
df_brand = session.sql("""
    SELECT BRAND_NAME, TOTAL_PRODUCTS, AVG_MRP, AVG_SELL_PRICE 
    FROM GOLD.BRAND_PERFORMANCE 
    ORDER BY AVG_SELL_PRICE DESC 
    LIMIT 10
""").to_pandas()

# Create Bar Chart
chart_brand = alt.Chart(df_brand).mark_bar().encode(
    x=alt.X('BRAND_NAME', sort=None, title='Brand'),
    y=alt.Y('AVG_SELL_PRICE', title='Average selling price'),
    color=alt.value('#1f77b4'),
    tooltip=['BRAND_NAME', 'TOTAL_PRODUCTS', 'AVG_SELL_PRICE']
).interactive()

st.altair_chart(chart_brand, use_container_width=True)

# ---------------------------------------------------------
# VISUALIZATION 2: Category Inventory Value
# ---------------------------------------------------------
st.header("2. Inventory Value by Category")
st.write("Total financial value of stock per category.")

# Query the Gold Table
df_cat = session.sql("""
    SELECT CATEGORY_NAME, TOTAL_INVENTORY_VALUE 
    FROM GOLD.CATEGORY_INVENTORY
    ORDER BY TOTAL_INVENTORY_VALUE DESC
""").to_pandas()

# Create Bar Chart
chart_cat = alt.Chart(df_cat).mark_bar().encode(
    x=alt.X('CATEGORY_NAME', sort=None, title='Category'),
    y=alt.Y('TOTAL_INVENTORY_VALUE', title='Total Inventory Value ($)'),
    color=alt.value('#ff7f0e'),
    tooltip=['CATEGORY_NAME', 'TOTAL_INVENTORY_VALUE']
).interactive()

st.altair_chart(chart_cat, use_container_width=True)

# ---------------------------------------------------------
# VISUALIZATION 3: Price Tier Analysis
# ---------------------------------------------------------
st.header("3. Price Tier Distribution")
st.write("Distribution of products across Budget, Mid-Range, and Premium tiers.")

# Query the Gold Table
df_tier = session.sql("""
    SELECT PRICE_TIER, PRODUCT_COUNT 
    FROM GOLD.PRICE_TIER_ANALYSIS
""").to_pandas()

# Create Bar/Donut Chart
base = alt.Chart(df_tier).encode(
    theta=alt.Theta("PRODUCT_COUNT", stack=True)
)

pie = base.mark_arc(outerRadius=120).encode(
    color=alt.Color("PRICE_TIER"),
    order=alt.Order("PRODUCT_COUNT", sort="descending"),
    tooltip=["PRICE_TIER", "PRODUCT_COUNT"]
)

text = base.mark_text(radius=140).encode(
    text="PRODUCT_COUNT",
    order=alt.Order("PRODUCT_COUNT", sort="descending"),
    color=alt.value("black") 
)

st.altair_chart(pie + text, use_container_width=True)

In [ ]:
-- select * from GOLD.PRICE_TIER_ANALYSIS
-- where PRICE_TIER = 'Unknown';

with cte as (
SELECT 
*,
    CASE 
        WHEN SELL_PRICE < 1000 THEN 'Budget'
        WHEN SELL_PRICE BETWEEN 1000 AND 3000 THEN 'Mid-Range'
        WHEN SELL_PRICE > 3000 THEN 'Premium'
        ELSE 'Unknown'
    END AS PRICE_TIER
FROM SILVER.FACT_FASHION
)

select * from cte where PRICE_TIER = 'Unknown'